# FIT5230 Week 4: Deepfakes I - Security Properties & Motion Models

## 1. Security Fundamentals: The CIA Triad

To understand how AI attacks systems, we must first understand the fundamental properties of security that are being violated.

### The Three Pillars
* **Confidentiality (CONF)**: Secrecy. Ensuring data is not leaked to unauthorized parties.
    * *Protection*: Encryption (transforming message $m$ to ciphertext $c$).
    * *AI Attack*: **Inference Attacks**. AI can find patterns in anonymized datasets (e.g., k-anonymity) to re-identify individuals or infer private attributes .
* **Integrity (INT)**: Data remains unchanged from its original source.
    * *Protection*: Hashing, Message Authentication Codes (MAC), Digital Signatures, Watermarking .
    * *AI Attack*: **Deepfakes**. AI generates fake content or tampers with existing content without detection .
* **Authentication (AUTH)**: Verifying that the source/origin is correct (User Identity).
    * *Protection*: Factors of authentication—Something you know (password), have (token), are (biometrics), or do (behavior) .
    * *AI Attack*: **Impersonation**. Deepfakes allow attackers to bypass biometric authentication (e.g., face swap) or social engineering checks .

---
<hr>

## 2. Deepfakes: attacking Integrity & Authentication

**Deepfakes** utilize generative AI to synthesize or manipulate media. They represent a direct attack on the Integrity (is this video real?) and Authentication (is this really Obama?) of media assets.

### Types of Deepfake Manipulation
1.  **Face Swap**: Transplantation of one identity onto another face. Attacks **AUTH** (Identity theft).
2.  **Facial Expression Transfer**: Transferring the expressions (and lip-sync) of a source actor onto a target actor. Attacks **INT** (Content tampering).
3.  **Motion Transfer / Puppet Master**: A source actor (driver) controls the head movement and expressions of a target actor. This is a holistic attack on both **INT** and **AUTH**.

---
<hr>

## 3. Technical Deep Dive: First Order Motion Model (FOMM)

**Paper**: Siarohin et al., NeurIPS 2019.

**Goal**: Image Animation. Generate a video sequence so that an object in a **Source Image ($S$)** is animated according to the motion of a **Driving Video ($D$)**.
* *Key Innovation*: It is **object-agnostic**. Once trained on a category (e.g., faces), it can animate any object in that category without specific prior knowledge of that object .

### Architecture Components
1.  **Keypoint Detector**: Unsupervised detection of important landmarks (keypoints) in both $S$ and $D$. It creates a heatmap of these points .
2.  **Dense Motion Module**: Maps the motion field from the Driving frame to the Source image using the detected keypoints.
3.  **Generation Module**: Warps the source image according to the dense motion field and inpaints occluded areas to produce the output.

### Mathematical Foundation: Optical Flow
To animate an image, the model must understand **Motion**, which is represented as **Optical Flow**.

**Concept**: Optical flow is the pattern of apparent motion of image objects between two consecutive frames. It is a vector field $(V_x, V_y)$ describing how each pixel $(x, y)$ moves to $(x+\Delta x, y+\Delta y)$ over time $\Delta t$.

**Brightness Constancy Assumption**:
The fundamental assumption is that the intensity (brightness) of a specific point on an object stays constant as it moves.
$$I(x, y, t) = I(x + \Delta x, y + \Delta y, t + \Delta t)$$
* $I(x, y, t)$: Pixel intensity at location $(x, y)$ at time $t$.
* Right side: The intensity of the *same point* at the new location and new time.

**Deriving the Optical Flow Equation:**
We use the **Taylor Series Expansion** to approximate the right side of the equation. The Taylor series allows us to estimate the value of a function near a point using its derivatives .
$$I(x+\Delta x, y+\Delta y, t+\Delta t) \approx I(x,y,t) + \frac{\partial I}{\partial x}\Delta x + \frac{\partial I}{\partial y}\Delta y + \frac{\partial I}{\partial t}\Delta t$$

Substituting this back into the Brightness Constancy Assumption ($I(x,y,t) = ...$), the $I(x,y,t)$ terms cancel out, leaving:
$$\frac{\partial I}{\partial x}\Delta x + \frac{\partial I}{\partial y}\Delta y + \frac{\partial I}{\partial t}\Delta t = 0$$

Dividing by $\Delta t$, we get the **Optical Flow Constraint Equation**:
$$\frac{\partial I}{\partial x}V_x + \frac{\partial I}{\partial y}V_y + \frac{\partial I}{\partial t} = 0$$

* **Conceptual Meaning**: The change in image brightness over time ($\frac{\partial I}{\partial t}$) is directly related to how fast the image structure (gradients $\frac{\partial I}{\partial x}, \frac{\partial I}{\partial y}$) is moving (velocity $V_x, V_y$). The model solves for $V_x$ and $V_y$ to understand how to warp the source image.

---
<hr>

## 4. Geometric Transformations (Warping)

Once motion is calculated, the source image must be **warped** (distorted/reshaped) to match the driving video's pose.

### Affine Transformations
Used for basic motion like translation, rotation, scaling, and shearing.
* **Properties**: Parallel lines remain parallel.
* **Matrix Form**:
    $$
    \begin{bmatrix} x \\ y \\ 1 \end{bmatrix} = 
    \begin{bmatrix} a_{11} & a_{12} & a_{13} \\ a_{21} & a_{22} & a_{23} \\ 0 & 0 & 1 \end{bmatrix} 
    \begin{bmatrix} u \\ v \\ 1 \end{bmatrix}
    $$
    * $(u, v)$: Original coordinates.
    * $(x, y)$: Warped coordinates.
    * The last row is always $[0, 0, 1]$ .

### Projective (Perspective) Transformations
Used for more complex motion where the depth/perspective changes (e.g., a face turning away).
* **Properties**: Parallel lines may converge (like train tracks in the distance).
* **Matrix Form**:
    $$
    \begin{bmatrix} x_w \\ y_w \\ w \end{bmatrix} = 
    \begin{bmatrix} a_{11} & a_{12} & a_{13} \\ a_{21} & a_{22} & a_{23} \\ a_{31} & a_{32} & a_{33} \end{bmatrix} 
    \begin{bmatrix} u \\ v \\ 1 \end{bmatrix}
    $$
    * The last row is **non-zero**. This introduces a scaling factor $w$.
    * To get the actual 2D coordinates, you must divide by $w$: $x = x_w / w$ and $y = y_w / w$. This division creates the non-linear "perspective" effect .

---
<hr>

## 5. Technical Deep Dive: Motion-supervised Co-Part Segmentation

**Paper**: Siarohin et al., ICPR 2021.

**Goal**: To transfer specific *parts* of a source image (e.g., hair, clothes) to a driving video, rather than just global motion .

**Problem**: Most segmentation requires expensive manual labels (supervised learning).
**Solution**: **Self-Supervised Learning** using Motion.
* **Assumption**: Parts of an object that move together belong to the same segment.
* **Training**:
    1.  Take two frames from the *same* video (Source and Target).
    2.  The network attempts to segment the object into $K$ parts + 1 background.
    3.  It predicts the **optical flow (motion)** for each segment.
    4.  It tries to **reconstruct** the Target frame by warping the Source frame using these segment-wise flows.
* **Logic**: If the network can successfully reconstruct the target frame, it proves that its segmentation and motion estimation were correct. The reconstruction loss ($L_{rec}$) acts as the supervisor .

# Deepfakes

## Security Properties
CIA triad:
1. CONFidentiality: protect information from accidental or intentional disclosure.  
    - Encryption  
2. INTegrity: protect information from accidental or intentional (malicious) modification.  
    - Digital Hash, Digital signature  
3. AVAILability: ensuring information is available to those who need it and when they need it (to authorized users).  
    - Load balancing, multi-cloud  

## AI attacks security
1. CONF: inference attacks vs privacy/CONF  
- putting data from different sources together to deduce classified information  
- k-anonymity: obscure more information so you can't deduce information so easily
- ℓ-diversity: have more diversity within same group  
- t-closeness: bucket data into larger groups to obscure details  

## Deepfakes: Motion Representation
In media, motion vectors can be combined with static images to create a video representation  
This can be applied to deepfakes to generate videos of a given image doing something else e.g. mona lisa in action   

## Deepfakes: Warping
Just like motion representation, gradually change one image to another via these transformations



# Tutorial
1. Outline the primary stages of a typical classification task. Discuss the importance of each stage in
achieving accurate pattern recognition.   
- Data collection  
- Data preprocessing  
- Training  

2. Consider a critical infrastructure company that outsources the training of its AI model, which is
designed to detect anomalies in sensor data. Describe how a malicious actor could leverage a
BadNet attack in this scenario. What immediate and long-term consequences could such an attack
have on the infrastructure?  
- Loss of trust  
- Failure of model  

3. Differentiate between an ”Untargetted attack” and a ”Targetted attack” within the context of
changing labels. Provide a unique real-world example for each type of attack.  
- Untargetted attack: make ChatGPT racist  
- Targetted attack: make algorithm hire you  

4. If a training dataset contains 50,000 samples and an attacker decides to poison 0.5% of these
samples with a backdoor, how many samples would be affected? Additionally, explain how an
attacker might manipulate parameters like ”step size” (learning rate), ”batch size,” and ”epochs” to
minimize their cost function during this attack.  
- Bigger batch size = more hidden, but harder to train it  
- More epoch = more prone to overfitting  

5. Compare and contrast the adversarial capabilities and limitations of an attacker in BadNet versus
TrojanNet. Highlight the key differences in their access to training data and model parameters.  
- Poisoned dataset vs extra model added  
- BadNet can be countered by adversarial training, but not TrojanNet  
- But TrojanNet can be countered by a code review  

6. Imagine you are a product manager at a company that develops autonomous vehicles, and your
team is considering purchasing pre-trained machine learning models from a third-party vendor.
What specific security risks would you be concerned about, and what initial verification steps
would you propose before integrating such models into your vehicles?  